In [ ]:
from mitmproxy import http, ctx
import time, re
import logging
import os

In [4]:
distr = "files/" #str(input("Directory: ")) #Директория для файлов
start_file = distr + "filtred" #str(input("File to save filtred stream to: ")) #Файл, в который пойдёт весь отфильтрованый поток
data_file = distr + "decoded" #str(input("Final file: ")) #Файл, куда будут сохранены нужные, раскодированные данные
script = "mitmdump_script.py" #Скрипт, при помощи которого достаём необходимые данные
with open ("save.txt", "w") as f: #Файл для передачи названия конечного файла скрипту
    f.write(data_file)

In [5]:
import threading

def StartMITMDump():
    # TODO change path to MITMdump
    os.system("mitmdump -w " + start_file + " '~t video/webm | ~t video/mp4'")
def StopMITMDump():
    time.sleep(int(input("Time to wait before stopping the flow: ")))
    os.system("pkill mitmdump")
def ReceiveData(script, file):
    # TODO change path to MITMdump
    os.system("mitmdump -s " + script + " -nr " + file)

In [56]:
# StartMITMDump(start_file) #Запускаем mitmdump (не включать mitmproxy, не использовать прокси сервер для локалки). 
# Собираем потоки в файл.
t = threading.Thread(target=StartMITMDump)
t.daemon = True
t.start()
time.sleep(240) #int(input("Time to wait before stopping the flow: ")))
os.system("pkill mitmdump")
# StopMITMDump() #Указываем через какое время в секундах остановить процесс mitmdump. Можно взять время с запасом
# (так как ещё есть реклама) и заниматься другими делами, предварительно отключив автовоспроизведение на ютубе.

[19:50:50.730] Only processing flows that match "~t video/webm | ~t video/mp4"
[19:50:50.740] HTTP(S) proxy listening at *:8080.
[19:50:55.724][127.0.0.1:41728] client connect
[19:50:55.758][127.0.0.1:41728] client disconnect
[19:50:55.762][127.0.0.1:41728] error establishing server connection: client disconnected
[19:50:55.791][127.0.0.1:41736] client connect
[19:50:55.818][127.0.0.1:41736] server connect i.ytimg.com:443 (142.251.1.119:443)
[19:50:56.825][127.0.0.1:41746] client connect
[19:50:56.835][127.0.0.1:41746] client disconnect
[19:50:56.836][127.0.0.1:41746] error establishing server connection: client disconnected
[19:50:56.852][127.0.0.1:41754] client connect
[19:50:56.889][127.0.0.1:41754] server connect www.youtube.com:443 (142.250.150.198:443)
[19:50:57.412][127.0.0.1:41770] client connect
[19:50:57.425][127.0.0.1:41776] client connect
[19:50:57.461][127.0.0.1:41776] server connect rr18---sn-n8v7knel.googlevideo.com:443 (173.194.180.100:443)
[19:50:57.464][127.0.0.1:4177

0

In [57]:
ReceiveData(script, start_file) #Записываем нужные данные в файл при помощи скрипта.

[19:54:50.442] Loading script mitmdump_script.py
files/decoded
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 446k
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 1.1k
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 595k
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 814k
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 1.4m
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 1.1k
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 2.0m
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplayback…
              << 200 OK 2.0m
127.0.0.1:41790: POST https://rr8---sn-n8v7knee.googlevideo.com/videoplay

In [7]:
d = []
itags = {"133":"240 px", "134":"360 px", "135": "480 px", "136": "720 px", "137": "1080 px", "160": "144 px", 
       "264": "1440 px", "298": "720 px", "299": "1080 px", "167": "360 px", "168": "480 px", "169": "720 px",
       "170" : "1080 px", "218": "480 px", "219": "144 px", "242": "240 px", "243": "360 px", "244": "480 px", 
       "245": "480 px", "246": "480 px", "247": "720 px", "248": "1080 px", "271": "1440 px", "278": "144 px",
       '399': '1080 px', '398': '720 px', '397': '480 px', '396': '360 px', '400': '1440 px', '401': '2160 px'} 
# Если встречаются неизвестные itag-и, можно найти, чему они соответствуют с помощью youtube-dl -F [ссылка на видео]
with open(data_file, "r") as f:
    a = [line.replace("\n","").split(",") for line in f.readlines()]
print(f"We made {len(a)} RESPs")
for idx, val in enumerate(a):
    ranges = val[3].split('-')
    if val[1] not in itags:
        print(f"BAD {val[1]} ITAG")
    d.append({"mime":val[0], "aitag":itags[val[1]], "duration":val[2], "range_start":ranges[0], "range_end": ranges[1], 
              "buffer":val[4], "clen":val[5],"first_request_byte":val[6], "request_complete": float(val[7]) / 1e6, 
              "first_response_byte":val[8], "response_complete":float(val[9]) / 1e6})
print(d[0])

We made 43 RESPs
{'mime': 'video/mp4', 'aitag': '480 px', 'duration': '203.970', 'range_start': '0', 'range_end': '209879', 'buffer': '0', 'clen': '17558745', 'first_request_byte': '1678725955.7618303', 'request_complete': 1678.7259557622674, 'first_response_byte': '1678725955.8831613', 'response_complete': 1678.725955933756}


In [64]:
import pandas as pd
names = {
    "mime":  str, 
    "aitag": str, 
    "duration": float,  #ms 
    "range_start": int, 
    "range_end": int, 
    "buffer": int, 
    "clen": int,  # byte
    "first_request_byte": float, 
    "request_complete": float, 
    "first_response_byte": float, 
    "response_complete": float
}    
df = pd.DataFrame(d).astype(names)

df["duration"]

,mime,aitag,duration,range_start,range_end,buffer,clen,first_request_byte,request_complete,first_response_byte,response_complete
0,video/mp4,480 px,203.97,0,209879,0,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
1,video/mp4,480 px,203.97,209880,562496,5157,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
2,video/mp4,480 px,203.97,562497,1395724,8909,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
3,video/mp4,480 px,203.97,1395725,2907928,16205,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
4,video/mp4,480 px,203.97,2907929,4625317,25302,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
5,video/mp4,720 px,203.97,0,457127,0,34187244,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
6,video/mp4,480 px,203.97,0,1175,0,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
7,video/mp4,720 px,203.97,457128,1066380,5505,34187244,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
8,video/mp4,480 px,203.97,562497,1395724,8909,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
9,video/mp4,480 px,203.97,1395725,2907928,16971,17558745,1.678726e+09,1.678726e+09,1.678726e+09,1.678726e+09
